# statdepth: An Interactive Guide

In this notebook we'll be exploring `statdepth,` a Python package for computing statistical depth of univariate functional data, multivariate functional data, and pointcloud data for distributions in $\mathbb{R}^d$

We'll begin by importing some libraries we may need

In [1]:
import numpy as np
import pandas as pd
from string import ascii_lowercase

from statdepth import FunctionalDepth, PointcloudDepth
from statdepth.testing import generate_noisy_pointcloud, generate_noisy_univariate

We'll now generate some random univariate functions with similar shape and some noise.

In [2]:
df = generate_noisy_univariate(data=[2,3,3.4,4,5,3.1,3,3,2]*3, columns=[f'f{i}' for i in range(20)])
df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19
0,0.077332,1.874619,1.891180,0.114874,0.261634,1.963706,1.563515,0.298318,0.586429,1.773932,0.712969,1.182490,0.377611,1.631300,0.815103,0.620301,1.773940,0.835447,1.219635,0.045879
1,0.115997,2.811929,2.836770,0.172311,0.392450,2.945559,2.345272,0.447478,0.879644,2.660898,1.069454,1.773734,0.566416,2.446949,1.222654,0.930451,2.660910,1.253170,1.829452,0.068818
2,0.131464,3.186853,3.215006,0.195285,0.444777,3.338301,2.657975,0.507141,0.996930,3.015684,1.212048,2.010232,0.641938,2.773209,1.385674,1.054512,3.015698,1.420259,2.073379,0.077994
3,0.154663,3.749239,3.782360,0.229747,0.523267,3.927413,3.127030,0.596637,1.172858,3.547864,1.425939,2.364979,0.755221,3.262599,1.630205,1.240602,3.547880,1.670893,2.439270,0.091758
4,0.193329,4.686549,4.727950,0.287184,0.654084,4.909266,3.908787,0.745796,1.466073,4.434829,1.782423,2.956224,0.944027,4.078249,2.037757,1.550752,4.434851,2.088617,3.049087,0.114697


Now we'll use our library to calculate band depth (using standard containment on $\mathbb{R}^2$

In [4]:
bd = FunctionalDepth([df], J=2, relax=False, quiet=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 18.31it/s]


Well, we can first look at the $n$ deepest and most outlying curves

In [8]:
bd.deepest(n=5)

6     0.473684
9     0.473684
10    0.463158
11    0.463158
17    0.442105
dtype: float64

In [9]:
bd.outlying(n=5)

13    0.178947
12    0.094737
14    0.094737
3     0.000000
15    0.000000
dtype: float64

But this is much more meaningful with visuals!

In [12]:
fig = bd.plot_deepest(n=3, return_plot=True, title='3 Deepest Curves, Plotted in Red')
fig.write_image('ex1_functional_deepest.pdf')

In addition to writing out the image in any general image format, we can visualize the results with Plotlys `.show()` method.

In [13]:
fig.show()

We can also plot the most outlying functions

In [16]:
fig = bd.plot_outlying(n=3, return_plot=True, title='3 Most Outlying Curves in Red')
fig.write_image('ex2_functional_depth.pdf')
fig.show()

Or, supposing we've tuned our FunctionalDepth to our liking, return our data with the `n` most outlying samples dropped

In [19]:
bd.get_deep_data(n=10).head()

,6,9,10,11,17,0,18,19,5,7
0,2.575717,2.844258,2.258476,3.055227,3.447407,2.194250,1.761308,3.477732,3.876017,1.078914
1,0.321965,0.355532,0.282309,0.381903,0.430926,0.274281,0.220163,0.434716,0.484502,0.134864
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.965894,1.066597,0.846928,1.145710,1.292778,0.822844,0.660490,1.304149,1.453506,0.404593


We can calculate depth for multivariate data using simplex depth, which generalizes the idea of containment in 2 dimensions to functions $f: D \rightarrow \mathbb{R}^n$, where $D$ is a set of discrete time indices.

In [25]:
from statdepth.testing import generate_noisy_multivariate

data = generate_noisy_multivariate(columns=list('ABC'), num_curves=10)

In [26]:
data[2]

,A,B,C
0,0.171008,0.607450,0.282832
1,0.626625,0.625463,0.604126
2,0.571434,0.135430,0.070607
3,0.554368,0.095617,0.026915
4,0.026602,0.189051,0.631318
5,0.436320,0.344728,0.141866
6,0.014896,0.053235,0.441031
7,0.290939,0.250025,0.288824
8,0.482558,0.689448,0.213720
9,0.251144,0.197549,0.279344


In [22]:
bd = FunctionalDepth(data, containment='simplex', relax=True, quiet=False)

Again, we can look at our curves ordered

In [12]:
bd.ordered()

9    0.952381
4    0.952381
5    0.880952
2    0.880952
6    0.722222
3    0.722222
8    0.444444
7    0.444444
1    0.000000
0    0.000000
dtype: float64

Now, let's try calculating band depth for some pointcloud data. Maybe you've sampled $n$ points from some distribution in $R^d$, and you'd like to understand which points are the most "central".

First, let's try this for some points sampled in $\mathbb{R^2}$

In [3]:
from statdepth import PointcloudDepth
from statdepth.testing import generate_noisy_pointcloud 

df = generate_noisy_pointcloud(n=50, d=2)
bd = PointcloudDepth(df, K=7, containment='simplex')
df.head()

NameError: name '_subsequences' is not defined

We can look at deepest points

In [14]:
bd.deepest(n=5)

39    0.200510
42    0.190816
31    0.164286
1     0.150000
20    0.147959
dtype: float64

Again, we can plot our data. Here, the lighter the color the deeper (more central) the point.

In [23]:
fig = bd.plot_depths(invert_colors=True, return_fig=True)

AttributeError: '_FunctionalDepthSeries' object has no attribute 'plot_depths'

We can also just plot the $n$ deepest points. 

In [16]:
bd.plot_deepest(n=5)

Or even the $n$ most outlying points, since often it's nice to know which data we should consider to be outliers

In [17]:
bd.plot_outlying(n=25)

But of course, if we're just defining depth using a certain measure of containment, there is no reason it shouldn't generalize to arbitrary dimensions. And indeed, this is the case. Let's take a look at some data in $\mathbb{R}^3$.

Notice, we're using sample depth because if we were to compute depth precisely, we'd be calculating about 500k simplices for each of our 50 datapoints, which can become unweildy fast.

However, it turns out that sample band depth is quite accurate for $K << n$, where $n$ is our number of datapoints, so this is definitely worth it.

In [18]:
df = generate_noisy_pointcloud(n=50, d=3, columns=list('ABC'))
bd = PointcloudDepth(df, K=7, containment='simplex')
df.head()

,A,B,C
0,0.771743,0.149359,0.028044
1,0.158763,0.315276,0.152181
2,0.264411,0.640676,0.936701
3,0.750667,0.399114,0.397765
4,0.921378,0.217658,0.024406


In [19]:
bd.deepest(n=5)

12    0.065306
24    0.065306
28    0.032653
25    0.032653
43    0.024490
dtype: float64

Well, looking at the 5 deepest points is interesting, but it's a lot more meaningful visually.

In [20]:
bd.plot_depths(invert_colors=True)

Or, we could just plot the $n$ deepest points

In [21]:
bd.plot_deepest(n=3)

In [22]:
bd.deepest(n=3)

12    0.065306
24    0.065306
28    0.032653
dtype: float64

In [23]:
bd.get_deep_data(n=5)

,A,B,C
12,0.354914,0.509508,0.743035
24,0.484699,0.253426,0.639967
28,0.225032,0.492542,0.654891
25,0.718285,0.379978,0.588705
43,0.543705,0.363351,0.804977


The above uses simplex containment, where to find the depth of a point in $\mathbb{R}^2$ we use all possible subsequences of 3 other points, construct a triangle, and check the proportion of triangles that our point is contained in.

We then to this for all other points we'd like to calculate depth for.

But this is not the only definition of depth. So let's use another below and see how it compares

In [24]:
bd = PointcloudDepth(df, K=3, containment='l1')

In [25]:
bd.plot_deepest(n=3)

Notice that this gives different deepest points than simplex depth. 

In [26]:
bd.get_deep_data(n=20)

,A,B,C
25,0.718285,0.379978,0.588705
24,0.484699,0.253426,0.639967
12,0.354914,0.509508,0.743035
43,0.543705,0.363351,0.804977
37,0.554620,0.192657,0.563665
6,0.367289,0.541579,0.315475
3,0.750667,0.399114,0.397765
41,0.531385,0.246896,0.303379
28,0.225032,0.492542,0.654891
29,0.196697,0.325023,0.514655


In [27]:
bd.drop_outlying_data(n=25)

,A,B,C
1,0.158763,0.315276,0.152181
2,0.264411,0.640676,0.936701
3,0.750667,0.399114,0.397765
6,0.367289,0.541579,0.315475
7,0.333215,0.052844,0.630215
9,0.764077,0.807100,0.558157
12,0.354914,0.509508,0.743035
13,0.665708,0.113994,0.566883
17,0.394223,0.249405,0.968503
19,0.893753,0.768649,0.404552
